**April 17, 2014**
 
**@author B.F. Habenicht (bfh)**


Script to extract MO coefficients, dipole matrix, and CI coefficients from Gaussian simulation and use these to calculate the transition dipole moments between all pairs of states. these will later be used by the python script TD_CIS.py to do excited state electron dynamics in an electric field. we first find the data in the gaussian log file. we then rotate the dipole matrix into the MO basis (from AO basis) using the MO coefficient matrix. once we have this, we then can multiply the appropriate CI coefficients together and multiply those terms times the dipole matrix in the MO basis. These terms are collected and summed to calculate the transition dipole moments between states.

**Important notes**

- For a CIS calculation the default window of MOs within which the single substitutions are carried out is determined by the heaviest atom in the system. To use all the MOs for a CIS calculation, set the following flag: `IOp(8/10=90)`.

\- KR, 06/07/2020

In [1]:
import sys
import numpy as np
import os
import shutil
import re
import time

**Defining conversion factors and parameters**

In [2]:
# dipole moment conversion factor: 1 debye = 0.393430307 a.u.
DtoAU = 0.39343
AUtoD = 1./DtoAU

# energy conversion factor: 1 a.u. = 27.211396 eV
AUtoEV = 27.211396
EVtoAU = 1./AUtoEV

# define square root of 2 - bfh
rt2 = np.sqrt(2)

In [3]:
#
# Route section for CIS calculation using "GDV i14+":
#   #P hf/sto-3g CIS(NStates=9,Root=1,AllTransitionDensities,Singlets)
#      nosymm IOp(3/33=3,3/36=1,4/33=3,5/33=3,5/72=-3,6/8=1,6/33=3,
#      8/10=90,9/33=3,9/40=3)
#
ci = 'cis'
molecule = 'sys1_h' # 'lih' or 'sys1_h' or 'heh+'
if (molecule == 'sys1_h'):
    mol = 'sys1_h'
    roots = 30
    cas = '-'+str(roots)+'roots'
    sN = 's0'
    basis = 'sto-3g'
elif (molecule == 'lih'):
    mol = 'lih'
    roots = 8
    cas = '-'+str(roots)+'roots'
    sN = 's0'
    basis = 'sto-3g'
elif (molecule == 'heh+'):
    mol = 'heh+'
    ci = 'cis'
    roots = 1
    cas = '-'+str(roots)+'roots'
    sN = 's0'
    basis = 'sto-3g'
ext = '.log'

Logfile = ci+cas+'_'+sN+'_'+mol+'_'+basis+ext # this is the string for '/path/to/file.log'
#Logfile = 'cis-4roots_s0_lih_sto-3g_nocore.log'
Logfile = './test_data/'+molecule+'/'+Logfile
#Logfile = '/path/to/Gaussian/log/file.log'

In [4]:
debug = 1
if debug == 1:
    print('file input:', Logfile)
    print('detecting whether file exists...')
    if os.path.isfile(Logfile) == True:
        print('file exists.')
    elif os.path.isfile(Logfile) == False:
        print('error: file does not exist.')
        quit()
debug = 0

file input: ./test_data/sys1_h/cis-30roots_s0_sys1_h_sto-3g.log
detecting whether file exists...
file exists.


In [5]:
# read the .LOG file after copying it to 'logfile.tmp'
outp = 'logfile.tmp'
shutil.copy(Logfile, outp)

infile = open(outp, 'r')
log_lines = infile.readlines()
log_lines = [re.sub(r'D','E', s) for s in log_lines]
infile.close()

**Defining/reading parameters**

In [6]:
# read parameters from the Gaussian .LOG file of CIS calculation
#  (currently works for restricted reference, singlet only)
#  - KR, 06/07/2020
for (n,line) in enumerate(log_lines):
    error = False
    try:
        if ('primitive gaussians' in line):
            elements = log_lines[n].split()
            # total number of basis fns/MOs with double occupancy
            NMOs = int(float(elements[0]))
        if ('NAtoms' in line):
            elements = log_lines[n].split()
            # total numer of atoms in the system
            NAtoms = int(float(elements[1]))
        if ('alpha electrons' in line):
            elements = log_lines[n].split()
            # number of alpha electrons in the system
            NELECT_A = int(float(elements[0]))
            # number of beta electrons in the system
            NELECT_B = int(float(elements[3]))
            # number of doubly occupied MOs in the reference
            NDOCC = NELECT_B
            # number of singly occupied MOs in the reference
            NSOCC = NELECT_A - NELECT_B
            NELECT = NELECT_A + NELECT_B
            NOCC = NSOCC + NDOCC
        # specific to CIS calculation
        if ('Range of M.O.s used for correlation' in line):
            elements = log_lines[n].split()
            NMOLow = int(float(elements[-2]))
            NMOHigh = int(float(elements[-1]))
            # number of MOs with frozen occupation
            #   NOTE: entire set of virtual MOs is considered for CIS
            NFRZ = NMOLow - 1
            NCAS = NMOHigh - NMOLow + 1
    except (ValueError, IndexError, TypeError, NameError):
        error = True
        print('Error encountered while reading parameters.')
        print(' Assigning them manually...')
        if (molecule == 'sys1_h'):
            NMOs=34
            NCAS=NMOs
            NFRZ=0
            NELECT=40
            NOCC=int(NELECT/2.)
            NAtoms=14
            NStates=30
        elif (molecule == 'lih'):
            NMOs=6
            NCAS=NMOs
            NFRZ=0
            NELECT=4
            NOCC=int(NELECT/2.)
            NAtoms=2
            NStates=9
        elif (molecule == 'heh+'):
            NMOs=2
            NCAS=NMOs
            NFRZ=0
            NELECT=2
            NOCC=int(NELECT/2.)
            NAtoms=2
            NStates=2
        break

# specific to CIS calculation
if (error == False):
    linenum = []
    for (n,line) in enumerate(log_lines):
        if ('Excited State  ' in line):
            elements = log_lines[n].split()
            linenum.append(n)
    count = -1
    n = linenum[count]
    elements = log_lines[n].split()[2].split(':')
    NStates = int(float(elements[0])) + 1     # number of CI states/CSFs

In [7]:
debug = 2
if debug == 1:
    print('CIS calculation with {} electrons and {} orbitals...'.format(NELECT, NCAS))
    print('total no. of MOs with double occupancy:\t\t {}'.format(NMOs))
    print('no. of occupied MOs with frozen configuration:\t {}'.format(NFRZ))
    print('total no. of occupied MOs:\t\t\t {}'.format(NOCC))
    print('total no. of atoms in the system:\t\t {}'.format(NAtoms))
    print('total no. of determinants in the system:\t {}'.format(NStates))
elif debug == 2:
    print('NMOs \t= {}'.format(NMOs))
    print('NCAS \t= {}'.format(NCAS))
    print('NFRZ \t= {}'.format(NFRZ))
    print('NELECT \t= {}'.format(NELECT))
    print('NOCC \t= {}'.format(NOCC))
    print('NAtoms \t= {}'.format(NAtoms))
    print('NStates = {}'.format(NStates))
debug = 0

NMOs 	= 34
NCAS 	= 34
NFRZ 	= 0
NELECT 	= 40
NOCC 	= 20
NAtoms 	= 14
NStates = 31


**Reading nuclear dipole moments**

In [8]:
# reading nuclear dipole moments from .LOG file
for (n,line) in enumerate(log_lines):
    try:
        if ('Nuclear    moments (au)' in line):
            elements = log_lines[n+1].split()
            # nuclear dipole moment along x-axis of the system in the input
            nuc_dipx = float(elements[1])
            # nuclear dipole moment along y-axis of the system in the input
            nuc_dipy = float(elements[2])
            # nuclear dipole moment along z-axis of the system in the input
            nuc_dipz = float(elements[3])
            nuc_dipx *= AUtoD
            nuc_dipy *= AUtoD
            nuc_dipz *= AUtoD
    except (ValueError, TypeError, NameError):
        print('Error encountered. Assigning nuclear dipole moments manually...')
        if (molecule == 'lih'):
            nuc_dipx=0.0
            nuc_dipy=0.0
            nuc_dipz=2.89128097
        elif (molecule == 'sys1_h'):
            nuc_dipx=0.0
            nuc_dipy=0.0
            nuc_dipz=0.0
        break

In [9]:
debug = 1
if debug == 1:
    print('Nuclear dipole moment along-')
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'\
          .format('x',nuc_dipx,nuc_dipx*DtoAU))
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'\
          .format('y',nuc_dipy,nuc_dipy*DtoAU))
    print('{}-axis: {:3.2f} D ({:3.4f} a.u.)'\
          .format('z',nuc_dipz,nuc_dipz*DtoAU))
debug = 0

Nuclear dipole moment along-
x-axis: 0.00 D (0.0000 a.u.)
y-axis: -0.00 D (-0.0000 a.u.)
z-axis: -0.00 D (-0.0000 a.u.)


**Reading dipole moment matrices represented in the AO basis**

In [10]:
dipX = np.zeros([NMOs,NMOs], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    if ('Multipole matrices IBuc=  518 IX=    1' in line):
        line_num.append(n)

loops = int(NMOs / 5) + 1
last = NMOs % 5
count = 0
n = line_num[count]
shift = 0
for k in range(loops):
    try:
        irange = NMOs - k*5
        for i in range(irange):
            if (k == (loops - 1)):
                if (i < last):
                    end = i + 1
                else:
                    end = last
            else:
                if (i <= 4):
                    end = i + 1
                else:
                    end = 5
            elements=log_lines[n+2+k+shift+i].split()
            for j in range(end):
                s = k*5 + j
                m = i + k*5
                dipX[m,s] = float(elements[j+1])
                if (i != j):
                    dipX[s,m] = dipX[m,s]
        shift += irange
    except (IndexError, ValueError):
        break

In [11]:
dipY = np.zeros([NMOs,NMOs], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    if ('Multipole matrices IBuc=  518 IX=    2' in line):
        line_num.append(n)

loops = int(NMOs / 5) + 1
last = NMOs % 5
count = 0
n = line_num[count]
shift = 0
for k in range(loops):
    try:
        irange = NMOs - k*5
        for i in range(irange):
            if (k == (loops - 1)):
                if (i < last):
                    end = i + 1
                else:
                    end = last
            else:
                if (i <= 4):
                    end = i + 1
                else:
                    end = 5
            elements=log_lines[n+2+k+shift+i].split()
            for j in range(end):
                s = k*5 + j
                m = i + k*5
                dipY[m,s] = float(elements[j+1])
                if (i != j):
                    dipY[s,m] = dipY[m,s]
        shift += irange
    except (IndexError, ValueError):
        break

In [12]:
dipZ = np.zeros([NMOs,NMOs], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    if ('Multipole matrices IBuc=  518 IX=    3' in line):
        line_num.append(n)

loops = int(NMOs / 5) + 1
last = NMOs % 5
count = 0
n = line_num[count]
shift = 0
for k in range(loops):
    try:
        irange = NMOs - k*5
        for i in range(irange):
            if (k == (loops - 1)):
                if (i < last):
                    end = i + 1
                else:
                    end = last
            else:
                if (i <= 4):
                    end = i + 1
                else:
                    end = 5
            elements=log_lines[n+2+k+shift+i].split()
            for j in range(end):
                s = k*5 + j
                m = i + k*5
                dipZ[m,s] = float(elements[j+1])
                if (i != j):
                    dipZ[s,m] = dipZ[m,s]
        shift += irange
    except (IndexError, ValueError):
        break

In [13]:
debug = 0
if debug == 1:
    print('Dipole X')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipX[i,j]))
    print('Dipole Y')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipY[i,j]))
    print('Dipole Z')
    for i in range(NMOs):
        for j in range(NMOs):
            print("%d  %d  %f" % (i,j, dipZ[i,j]))
debug = 0

**Reading MO coefficients as column-vectors**

In [14]:
# specific to CIS calculation
MO = np.zeros([NMOs,NMOs], np.float64)

line_num = []
for (n, line) in enumerate(log_lines):
    try:
        if ('Alpha MOs' in line):
            line_num.append(n)
    except (IndexError, ValueError):
        pass

count = -1
nline = line_num[count]
loops = int(NMOs / 5) + 1
last = NMOs % 5
for k in range(loops):
    for i in range(NMOs):
        try:
            if (k == (loops - 1)):
                end = last
            else:
                end = 5
            dum1 = nline+3+k*(2+NMOs)+i
            elements = log_lines[dum1].split()
            for j in range(end):
                s = k*5 + j
                m = i
                MO[m,s] = float(elements[j+1])
        except (IndexError, ValueError):
            break

In [15]:
debug = 0
if debug == 1:
    print('MO coefficients (column-vectors):')
    print(MO)
debug = 0

**Transforming the dipole moment matrices from AO basis to MO basis**

*(Not necessary for the current method which works entirely within the AO basis)*

In [16]:
dipXMO = MO.T.dot(dipX).dot(MO)
dipYMO = MO.T.dot(dipY).dot(MO)
dipZMO = MO.T.dot(dipZ).dot(MO)

In [17]:
debug = 0
if (debug == 2):
    print('MO Dipole:')
    print('x:\n',dipXMO)
    print('y:\n',dipYMO)
    print('z:\n',dipZMO)
debug = 0

**Determining the dimensions of the matrix of CI vectors**

**NOTE**: The CIS vectors will be, in general, arranged in a rectangular matrix of dimensions $(M \times N)$, where $M$=`NStates` and $N$=`(NOCC-NMOLow+1)*(NMOHigh-NOCC)]` (or vice versa). For convenience, CI vectors will be saved in arrays as row-vectors. The total number of CIS states is `NStates`, however, the ground state does not mix with the excited states (formed from singly-subsituted determinants upon the reference state). Therefore, the CI vector of the ground state is assigned a vector with full weight to the Hartree-Fock determinant (element `[0,0]` of `CI_vect`).

In [18]:
# specific to CIS calculation
M = NStates
N = int((NOCC - NMOLow + 1)*(NMOHigh - NOCC)) + 1

# forming an array of compound indices. This array will be used as
#   reference to determine index of the singly-substituted determinant
#   within the CI vector array
cmpd_indx = []
cmpd_indx_dict = {}
k = 1
for i in range((NMOLow-1),NOCC):
    for a in range(NOCC,NMOHigh):
        ia = int((a+1)*((a+1)+1)/2 + (i+1))
        cmpd_indx.append(ia)
        cmpd_indx_dict[str(k)] = [i, a, ia]
        k += 1

In [19]:
debug = 0
if debug == 1:
    print('M = {}; N = {}'.format(M,N))
elif debug == 2:
    print('M = {}; N = {}'.format(M,N))
    print('list of compound indices used:')
    print(cmpd_indx)
    print(cmpd_indx_dict)
debug = 0

**Reading CI coefficients and energies**

In [20]:
# specific to CIS calculation
CI_vect = np.zeros([M,N], np.float64)
CI_E = np.zeros([M], np.float64)

# first element of CI_E will be the HF energy
for (n, line) in enumerate(log_lines):
    if ('E(RHF) =' in line):
        elements = log_lines[n].split()
        CI_E[0] = float(elements[4])

CI_vect[0,0] = np.float64(1)    # HF determinant contribution to GS is 1.0
ia = 0
for (n, line) in enumerate(log_lines):
    try:
        if ('Excited State  ' in line):
            elements = log_lines[n].split()[2].split(':')
            st_ci = int(float(elements[0]))
            CI_E[st_ci] = float(log_lines[n].split()[4])*EVtoAU + CI_E[0]
            for i in range(N):
                try:
                    elements = log_lines[n+1+i].split('->')
                    mo_i = int(float(elements[0]))
                    mo_a = int(float(elements[1].split()[0]))
                    c_ia = float(elements[1].split()[1])
                    ia = int(mo_a*(mo_a+1)/2 + mo_i)
                    CI_vect[st_ci,(cmpd_indx.index(ia)+1)] = c_ia*rt2
                except (ValueError, IndexError, TypeError):
                    break
    except (IndexError):
        break

In [21]:
debug = 0
if debug == 1:
    print('# CI states = ',len(CI_E))
    print('CI Eigenvalues (eV):')
    print(CI_E[:]*AUtoEV)
elif debug == 2:
    print('CIS Eigenvalues and Eigenvectors:')
    for i in range(M):
        print('ECIS(S{}) = {: 4.3f} eV'.format(i, CI_E[i]*AUtoEV))
        print(CI_vect[i,:])
debug = 0

**Reading MO configurations for each determinant in the basis**

**NOTE**: The occupation numbers (which is what comprises the "MO configurations") are calculated as a sum of weighted occupations of each orbital in the active space, the weights corresponding to the norm-squared of the CI coefficients.

In [22]:
# specific to CIS calculation

# initialize arrays to read and analyze MO configuration for each configuration - bfh
config = np.zeros((M,NCAS), np.float64)

for i in range(M):
    for j in range(NCAS):
        if ((j+NFRZ) < NOCC):
            config[i,j] = 2
        else:
            config[i,j] = 0
    if (i > 0):
        for j in cmpd_indx_dict:
            indx = int(j)
            index_i = cmpd_indx_dict[str(j)][0]
            index_a = cmpd_indx_dict[str(j)][1]
            index_ia = int((index_a+1)*((index_a+1)+1)/2 + (index_i+1))
            occ_i = 1*CI_vect[i,indx]*np.conjugate(CI_vect[i,indx])
            occ_a = 1*CI_vect[i,indx]*np.conjugate(CI_vect[i,indx])
            config[i,(index_i-NMOLow+1)] -= occ_i
            config[i,(index_a-NMOLow+1)] += occ_a

In [23]:
debug = 0
if (debug == 1):
    print('CIS configurations (within the active space):')
    for i in range(NStates):
        print('CI state {}   '.format(i))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config[i,j],end='\n')
            else:
                print(config[i,j],end=' ')
elif (debug == 2):
    print('CIS configurations and multiplicities:')
    for i in range(NStates):
        print('CI state {}   (multiplicity = {})   '.format(i,multi[i]))
        for j in range(NCAS):
            if (j == (NCAS - 1)):
                print(config[i,j],end='\n')
            else:
                print(config[i,j],end=' ')    
debug = 0

**Calculating electric dipole moment matrix in the CI state basis**

**NOTE**: No calculation is involved for CIS transition dipole moment matrix elements, as these values are printed in the `Gaussian` output (`.LOG`) file. For the state dipole moments, we use the formula: $\boldsymbol{\mu}_\text{CI}^{i} = \text{tr}(\textbf{P}_\text{CI}^\text{AO} \boldsymbol{\mu}_\text{AO}^{i})$, where, $i \in \{x, y, z\}$

In [24]:
# specific for CIS calculation
dipXCI = np.zeros([M,M], np.float64)
dipYCI = np.zeros([M,M], np.float64)
dipZCI = np.zeros([M,M], np.float64)

densCI_AO = np.zeros((NMOs, NMOs), np.float64)

print('calculating CI state dipole moments...')

line_num = []
for (n, line) in enumerate(log_lines):
    # reading ground state (HF) density
    if ('Alpha density for excited state' in line):
        st1 = int(float(log_lines[n].split()[-1]))
        loops = int(NMOs / 5) + 1
        last = NMOs % 5
        shift = 0
        for k in range(loops):
            try:
                irange = NMOs - k*5
                for i in range(irange):
                    if (k == (loops - 1)):
                        if (i < last):
                            end = i + 1
                        else:
                            end = last
                    else:
                        if (i <= 4):
                            end = i + 1
                        else:
                            end = 5
                    elements=log_lines[n+2+k+shift+i].split()
                    for j in range(end):
                        s = k*5 + j
                        m = i + k*5
                        densCI_AO[m,s] = float(elements[j+1])
                        if (i != j):
                            densCI_AO[s,m] = densCI_AO[m,s]
                shift += irange
            except (IndexError, ValueError):
                break
        densCI_AO *= 2
        dipXCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipX))
        dipYCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipY))
        dipZCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipZ))
    # reading CI state density
    elif ('Final density matrix:' in line):
        st1 = 0
        loops = int(NMOs / 5) + 1
        last = NMOs % 5
        shift = 0
        for k in range(loops):
            try:
                irange = NMOs - k*5
                for i in range(irange):
                    if (k == (loops - 1)):
                        if (i < last):
                            end = i + 1
                        else:
                            end = last
                    else:
                        if (i <= 4):
                            end = i + 1
                        else:
                            end = 5
                    elements=log_lines[n+2+k+shift+i].split()
                    for j in range(end):
                        s = k*5 + j
                        m = i + k*5
                        densCI_AO[m,s] = float(elements[j+1])
                        if (i != j):
                            densCI_AO[s,m] = densCI_AO[m,s]
                shift += irange
            except (IndexError, ValueError):
                break
        densCI_AO *= 2
        dipXCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipX))
        dipYCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipY))
        dipZCI[st1,st1] = -1*np.trace(np.matmul(densCI_AO,dipZ))

calculating CI state dipole moments...


In [25]:
# specific to CIS calculation

print('reading CI transition dipole moments...')

#
# calculating CI transition dipole moments
#
for (n, line) in enumerate(log_lines):
    if ('Excited to excited state transition electric dipole moments' in line):
        for i in range(int((M-2)*(M-1)/2)):
            try:
                elements = log_lines[n+3+i].split()
                st1 = int(float(elements[0]))
                st2 = int(float(elements[1]))
                dipXCI[st1,st2] = float(elements[2])
                dipXCI[st2,st1] = dipXCI[st1,st2]
                dipYCI[st1,st2] = float(elements[3])
                dipYCI[st2,st1] = dipYCI[st1,st2]
                dipZCI[st1,st2] = float(elements[4])
                dipZCI[st2,st1] = dipZCI[st1,st2]
            except (IndexError, ValueError):
                break
    elif ('Ground to excited state transition electric dipole' in line):
        st1 = 0
        for i in range(1,M):
            try:
                elements = log_lines[n+1+i].split()
                st2 = int(float(elements[0]))
                dipXCI[st1,st2] = float(elements[1])
                dipXCI[st2,st1] = dipXCI[st1,st2]
                dipYCI[st1,st2] = float(elements[2])
                dipYCI[st2,st1] = dipYCI[st1,st2]
                dipZCI[st1,st2] = float(elements[3])
                dipZCI[st2,st1] = dipZCI[st1,st2]
            except (IndexError, ValueError):
                break

reading CI transition dipole moments...


In [26]:
debug = 0
if debug == 1:
    print('CI dipole moment matrices (a.u.):')
    print('along x-axis:\n', dipXCI)
    print('along y-axis:\n', dipYCI)
    print('along z-axis:\n', dipZCI)
elif debug == 2:
    for i in range(NStates):
        print('dipole moment (z) of CI state {}  = {} a.u.  '\
              .format(i+1, dipZCI[i,i]))
debug = 0

**Writing read and computed CI data to `tdm_tdcis.txt` for TDCI calculation** 

In [27]:
# this part is written by bfh and is mostly unchanged

# have now calculated the Transition dipole moments. time to get atom coordinates and dump data
#  to file so TD_CIS.py can read it in. - bfh

outname = 'tdm_tdci.txt' # name of the output file, to be read by TDCI script

tdm = open(outname, 'w')
tdm.write('===========================================================================================\n')
tdm.write('   This file contains the information one needs to run a time-dependent CIS simulation.    \n')
tdm.write('===========================================================================================\n')
tdm.write('An initial Gaussian calculation was used to calculate the CI eigenvalues and eigenvectors  \n')
tdm.write('as well as the CI state configurations. The program calc_tdm.py was then used to calculate \n')
tdm.write('the transition dipole moments between the CI states as well as the project of the CI states\n')
tdm.write('onto the molecular orbitals. This file was generated by calc_tdm.py as input to TD_CIS.py. \n')
tdm.write(' -BFH 9 May 2014 (modified by KR 8 June 2019)                                              \n')

tdm.write('\n\n')

2

In [28]:
# parameters for the TDCI script

tdm.write('CI calculation parameters\n\n')
tdm.write(' title   = {}\n'.format(mol+'_'+basis))
tdm.write(' method  = {}\n'.format(ci))
tdm.write(' NMOLow  = {}\n'.format(NMOLow))
tdm.write(' NMOHigh = {}\n'.format(NMOHigh))
tdm.write(' NMOs    = {}\n'.format(NMOs))
tdm.write(' NCAS    = {}\n'.format(NCAS))
tdm.write(' NFRZ    = {}\n'.format(NFRZ))
tdm.write(' NOCC    = {}\n'.format(NOCC))
tdm.write(' NELECT  = {}\n'.format(NELECT))
tdm.write(' NAtoms  = {}\n'.format(NAtoms))
tdm.write(' NStates = {}\n'.format(NStates))

tdm.write('\n\n')

2

In [29]:
tdm.write('Nuclear dipole moments (a.u.) \n\n')
tdm.write(' x = {: 4.3f} \n'.format(nuc_dipx*DtoAU))
tdm.write(' y = {: 4.3f} \n'.format(nuc_dipy*DtoAU))
tdm.write(' z = {: 4.3f} \n'.format(nuc_dipz*DtoAU))

tdm.write('\n\n')

2

In [30]:
tdm.write('Setting up the atomic coordinates \n\n')
tdm.write(' Standard orientation: \n')
tdm.write(' ------------------------------------------------------------------- \n')
tdm.write(' Center   Atomic     Atomic            Coordinates (Angstroms)       \n')
tdm.write(' Number   Number      Type            X            Y          Z      \n')
tdm.write(' ------------------------------------------------------------------- \n')


coords = np.zeros([NAtoms,3], np.float64)
atom_info = np.zeros([NAtoms,3], np.float64)

for (n, line) in enumerate(log_lines):
    if (' Standard basis:' in line):
        for i in range(NAtoms):
            elements = log_lines[n + i + 6].split()
            atom_info[i,0] = float(elements[0])
            atom_info[i,1] = float(elements[1])
            atom_info[i,2] = float(elements[2])
            coords[i,0] = float(elements[3])
            coords[i,1] = float(elements[4])
            coords[i,2] = float(elements[5])
            tdm.write('  %-10s %-10s %-10s %-10s %-10s %-10s' % (atom_info[i][0], atom_info[i][1], atom_info[i][2], coords[i][0], coords[i][1], coords[i][2]) + "\n")

tdm.write('\n\n')

2

In [31]:
# input SCF energy- assume energy of first NState - bfh

tdm.write('SCF Energy \n\n')
tdm.write(' SCF Done: E(RHF) = ' + str(CI_E[0]) + '  Ha \n')

tdm.write('\n\n')

# Excited State energies - bfh

tdm.write('Excited state energies (eV) \n\n')
for i in range(1,NStates):
    prt_E = (CI_E[i] - CI_E[0])*AUtoEV
    tdm.write(' Excited State {:3d} \t {:4.6f}\n'.format(i, prt_E))

tdm.write('\n\n')

2

In [32]:
# CI electric dipole moments - bfh

tdm.write('CI electric dipole moments (a.u.) \n\n')
tdm.write(' CI state and transition dipole moments\n')
tdm.write(' state I  state J \t      X \t      Y \t      Z \t Dip. S.  Osc.\n' )
for i in range(M):
    for j in range(M):
        tdm.write('   {} \t    {} \t\t  {:3.7f}  \t  {:3.7f}  \t  {:3.7f}  \t  {}  \t  {}\n'.format(i, j, dipXCI[j,i], dipYCI[j,i], dipZCI[j,i], 0.0, 0.0))

tdm.write('\n\n')

2

In [33]:
# Finally, need to write out information on CI configs and MOs occupations - bfh
tdm.write('Output CI Vectors \n\n')
tdm.write(' CI state       CI coeffs 1...N \n')
for i in range(M):
    tdm.write('  %4d \t\t' % i )
    for j in range(N):
        tdm.write('%6.6f\t' % CI_vect[i,j])
    tdm.write('\n')

tdm.write('\n\n')

#note, want state # then number for each orbital. something like
#   state       MO 1     MO 2    MO 3     MO 4
#     1         1.95     1.95    0.05     0.05
# - bfh

mo_weight = np.zeros([NCAS], np.float64)
tdm.write('Orbital electron configuration weighting for CI States \n\n')
tdm.write(' CI State \t\t Orbital Weights 1...N  \n')
for i in range(M):
    for k in range(NCAS):
        mo_weight[k] = float(config[i,k])
    tdm.write('  {:3d}\t\t'.format(i))
    for k in range(NCAS):
        if (k == (NCAS - 1)):
            tdm.write('{:4.6f}\n'.format(mo_weight[k]))
        else:
            tdm.write('{:4.6f}\t'.format(mo_weight[k]))
        mo_weight[k] = 0

tdm.write('\n\n')

tdm.close()